In [1]:
import os
import json
import torch
import numpy as np
import cv2
from tqdm import tqdm

os.chdir('../')
os.getcwd()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/workspace'

In [3]:
from augmentation import RoMixGen_Img, RoMixGen_Txt
import random
from PIL import Image 
from torchvision import transforms 

In [4]:
transform_after_mix = transforms.Compose([
                                            transforms.RandomResizedCrop(256,scale=(0.5, 1.0), interpolation=Image.BICUBIC),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
                                        ])

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:891: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [5]:
RoMixGen_Img = RoMixGen_Img(image_dict = json.load(open('data/COCO/Annotations/coco_img_info_0.01_bbox_center_0.7.json')),
                            image_root = 'data/COCO/Images_0.01_bbox_center_0.7',
                            transform_after_mix = transform_after_mix,
                            resize_ratio = 2)
RoMixGen_Txt = RoMixGen_Txt(image_caption = json.load(open('data/COCO/Annotations/coco_img_info_0.01_bbox_center_0.7.json')))

In [6]:
with open ('data/COCO/Annotations/coco_objbg_info_0.01_bbox_center_0.7.json') as f:
    objbg_dict = json.load(f)
with open ('data/COCO/Annotations/coco_img_info_0.01_bbox_center_0.7.json') as f:
    image_dict = json.load(f)
with open ('data/COCO/Annotations/coco.json') as f:
    original_dataset = json.load(f)

In [7]:
# make 123,287 romixed images and captions
# new image id start from 000001 ~ 123287
# save image to data/COCO/pre_romix_coco_0.01_bbox_center_0.7
# save caption to data/COCO/Anotations/pre_romix_caption_0.01_bbox_center_0.7.json
total_imagenum = 123287
obj_ids = objbg_dict['obj']
bg_ids = objbg_dict['bg']

In [8]:
classic_dataset = []
new_dataset = {}

if not os.path.exists('data/COCO/pre_romix_coco_0.01_bbox_center_0.7'):
    os.makedirs('data/COCO/pre_romix_coco_0.01_bbox_center_0.7')

for img_id in tqdm(range(total_imagenum)):
    # Keep retrying until the image generation succeeds
    while True:
        obj_id, bg_id = str(random.choice(obj_ids)), str(random.choice(bg_ids))
        try:
            img = RoMixGen_Img(obj_id, bg_id)
            break # exit the loop if image generation succeeds
        except:
            continue

    txt = RoMixGen_Txt(obj_id, bg_id)
    img = torch.permute(img, (1,2,0)).detach().numpy()
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    img = img*255

    # Write image to disk
    img_path = os.path.join('data/COCO/pre_romix_coco_0.01_bbox_center_0.7', f'{img_id:06d}.jpg')
    if not os.path.exists(img_path):
        cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Add captions to datasets
    classic_dataset.extend([{'caption': _, 'image': img_path, 'image_id': f"coco_{img_id:06d}"} for _ in txt])
    new_dataset.setdefault(str(img_id), []).extend(txt)

    # dump datasets
    if img_id % 100 == 0:
        with open('data/COCO/Annotations/pre_romix_caption_0.01_bbox_center_0.7.json', 'w') as f:
            json.dump(classic_dataset, f)
        with open('data/COCO/Annotations/pre_romix_caption_new_0.01_bbox_center_0.7.json', 'w') as f:
            json.dump(new_dataset, f)


  0%|          | 0/123287 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 288/123287 [04:40<33:14:03,  1.03it/s]


KeyboardInterrupt: 